In [4]:
import pandas as pd 
from gensim import corpora, models, similarities

In [5]:
review = pd.read_pickle('../output/bar_reviews_cleaned_and_tokenized.pickle')

## Now we must generate a dictionary which maps vocabulary into a number

In [29]:
import time 
from itertools import chain



print 'Generating vector dictionary....'
review_flatten = list(chain.from_iterable(review.cleaned_tokenized.iloc[:]))
start = time.time()
id2word_wiki = corpora.Dictionary(review_flatten)
print 'Dictonary generated in %1.2f seconds'%(time.time()-start)


Generating vector dictionary....
Dictonary generated in 45.46 seconds


In [30]:
# Convert corpus to bag of words for use with gensim...
# See https://radimrehurek.com/gensim/tut1.html#from-strings-to-vectors
corpus = map(lambda doc: id2word_wiki.doc2bow(doc), review_flatten)
corpora.MmCorpus.serialize('../output/bar_corpus.mm', corpus)


# Can load the corpus with 
# from gensim import corpora
# corpus = corpora.MmCorpus('../output/bar_corpus.mm')



In [45]:

import gensim
print 'Fitting LDA Model'
start = time.time()
ldamodel = gensim.models.ldamodel.LdaModel(corpus[:2000], num_topics=10, 
                                           id2word=id2word_wiki, passes=5,)
print 'LDA fit in %1.2f seconds'%(time.time()-start)

Fitting LDA Model
LDA fit in 1.68 seconds


In [46]:
for topic in ldamodel.print_topics(num_topics=3, num_words=3): 
    print topic


(0, u'0.002*very + 0.001*sandwich + 0.001*place')
(6, u'0.000*wing + 0.000*nice + 0.000*checked')
(8, u'0.000*sauce + 0.000*nice + 0.000*also')


In [50]:
from sklearn.decomposition import LatentDirichletAllocation, nmf


lda = LatentDirichletAllocation(n_topics=10, evaluate_every=1000, n_jobs=12, verbose=True)

lda.fit(corpus[:2000])

/home/carlson/anaconda/envs/insight/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: setting an array element with a sequence.